# Language Learning Chatbot with LLMs
This notebook demonstrates a chatbot built using OpenAI's LLMs and LangChain.

### Features:
- Language selection
- Scene-based conversation
- Mistake correction and tracking
- Feedback summary at the end

We use SQLite to log user mistakes and provide personalized feedback.

In [ ]:
# Install required packages
!pip install openai langchain sqlite3

## 1. Import Required Libraries

In [ ]:
import openai
from langchain.chat_models import ChatOpenAI
from langchain.prompts import ChatPromptTemplate
import sqlite3
import os

Next steps will include setting up the chatbot logic, conversation handling, mistake tracking, and final feedback.

## 2. Initialize Database for Mistake Logging

In [ ]:
# Initialize SQLite database to store mistakes
conn = sqlite3.connect('mistakes.db')
cursor = conn.cursor()

# Create table
cursor.execute('''
CREATE TABLE IF NOT EXISTS mistakes (
    id INTEGER PRIMARY KEY AUTOINCREMENT,
    user_input TEXT,
    correction TEXT
)''')
conn.commit()

## 3. Setup OpenAI Chat Model

In [ ]:
# Provide your OpenAI key here (REMOVE before submission)
openai.api_key = 'YOUR_OPENAI_API_KEY'
llm = ChatOpenAI(temperature=0.7)

## 4. Define Helper Functions for Chat and Mistake Handling

In [ ]:
def log_mistake(user_input, correction):
    cursor.execute("INSERT INTO mistakes (user_input, correction) VALUES (?, ?)", (user_input, correction))
    conn.commit()

def get_mistake_summary():
    cursor.execute("SELECT * FROM mistakes")
    rows = cursor.fetchall()
    return rows


## 5. Start Language Learning Chatbot

In [ ]:
# User input setup
known_lang = input("What language do you know? ")
learning_lang = input("What language do you want to learn? ")
level = input("What is your current level (beginner/intermediate/advanced)? ")
scene = input("Pick a scene for conversation (e.g., cafe, airport, market): ")

print(f"\nStarting conversation in {learning_lang} at a {scene} scenario.")

# Prompt setup
prompt_template = ChatPromptTemplate.from_template(
    "You are a helpful language tutor speaking to someone who knows {known_lang} and is learning {learning_lang}. "
    "Their level is {level}. You are in a {scene}. Speak only in {learning_lang}. If the user makes a mistake, "
    "respond with correction, and keep a note of it."
)

chat_history = []
for _ in range(5):
    user_input = input("You: ")
    formatted_prompt = prompt_template.format_messages(
        known_lang=known_lang, learning_lang=learning_lang, level=level, scene=scene
    ) + chat_history + [user_input]
    
    response = llm.predict_messages(formatted_prompt)
    print("Bot:", response.content)

    if "Correction:" in response.content:
        parts = response.content.split("Correction:")
        correction = parts[1].strip()
        log_mistake(user_input, correction)

    chat_history.append(("user", user_input))
    chat_history.append(("bot", response.content))

## 6. Review Mistakes and Give Feedback

In [ ]:
# Show user their mistakes
mistakes = get_mistake_summary()
print("\nHere are the mistakes you made:")
for i, (id, user_input, correction) in enumerate(mistakes):
    print(f"{i+1}. You said: '{user_input}' -> Correction: '{correction}'")

print("\nFocus on the grammar and vocabulary corrections above to improve!")